In [1]:
import pandas as pd
import yfinance as yf
import funzioni as fx
import plotly.offline as pyo
import numpy as np
import os

from tensorflow.keras.models import load_model
from joblib import dump

pd.set_option("display.max_columns", None)

In [2]:
learning_rate = 0.001
batch_size = 1
look_back = 60
file_modello = "LSTM_prezzo.keras"

In [3]:
elenco_usin = pd.read_parquet("Tickers_De_Giro.parquet")
elenco_usin = elenco_usin.loc[elenco_usin["Ticker"] == "BTG", ["USIN", "Ticker"]]
elenco_usin = elenco_usin.sample(frac=1)
tot_usin = len(elenco_usin)
i = 1
if os.path.exists(file_modello):
    model = load_model(file_modello)
    print("Modello esistente")
else:
    model = 0
    print("Nuovo modello")
for index, riga in elenco_usin.iterrows():       
    print(f"\r{i} di {tot_usin}) Caricamento ticker {riga['Ticker']}                           \n", end=' ', flush=True)
    ticker = yf.download(riga['USIN'], start='2005-01-01', end='2019-07-31', progress=False)
    if len(ticker) < 500:
        continue
    else:
        ticker = fx.crea_indicatori(ticker)
        features, target = fx.features_e_target_regressione(ticker)
        ticker["Target"] = target
        print(f"\r{i} di {tot_usin}) Addestramento ticker {riga['Ticker']}                           \n", end=' ', flush=True)
        try:
            model, X_scaler, Y_scaler = fx.addestramento_regressione(model, features=features, target=target, learning_rate=learning_rate, batch_size=batch_size, look_back=look_back) 
            model.save(file_modello)
            dump(X_scaler, f'Scalers/{riga["Ticker"]}_X_scaler.joblib') 
            dump(Y_scaler, f'Scalers/{riga["Ticker"]}_Y_scaler.joblib') 
        except Exception as e:
            print(e)
            continue
        i += 1

print("fine addestramento")

Nuovo modello
1 di 1) Caricamento ticker BTG                           
1 di 1) Addestramento ticker BTG                           
 Epoch 1/5
2093/2093 [==============================] - 67s 29ms/step - loss: 0.0709 - val_loss: 0.0177
Epoch 2/5
2093/2093 [==============================] - 59s 28ms/step - loss: 0.0300 - val_loss: 0.0261
Epoch 3/5
2093/2093 [==============================] - 61s 29ms/step - loss: 0.0283 - val_loss: 0.0250
Epoch 4/5
2093/2093 [==============================] - 59s 28ms/step - loss: 0.0270 - val_loss: 0.0204
Epoch 5/5
17/17 [==============================] - 2s 18ms/step - loss: 0.0172
Loss: 0.01722344011068344
fine addestramento
